In [1]:
import geneMGF
import pickle
import mgf_math
import sympy
import importlib
importlib.reload(mgf_math)
importlib.reload(geneMGF)
sympy.init_printing()

A useful quantitiy for understanding how genetic architecture impacts the trait distribution is the coefficient of variation of the variance. This is defined as 
$$
\frac{\sqrt{Var[V_g]}}{E[V_g]}.
$$
Since we already have an expression for the expected variance in the population, what we need to derive is the variance of the variance.
$$
  Var[V_g] =  Var\left[ \frac{1}{N}\sum \left( Y_i - \frac{\sum Y_j}{N} \right)^2\right] \\
  = \frac{1}{N^2}\left( \sum Var[(Y_i - \frac{\sum Y_j}{N})^2] + 2 \sum_{i \neq j} 
              Cov[(Y_i - \frac{\sum Y_k}{N})^2, (Y_j - \frac{\sum Y_k}{N})^2]\right)
$$

The first term can be ignored if the population size is large. The second term is determined by
the covariance in the squared deviations from the mean. The covariance can be calculated by
considering its individual parts.

\begin{align*}
  &Cov[Y_i^2, Y_j^2] \approx -2Cov[Y_i^2,Y_j^2] \nonumber \\
  &Cov[Y_i^2, -2Y_j \frac{\sum Y_k}{N}] \approx -2Cov[Y_i^2,Y_jY_k] \nonumber \\
  &Cov[Y_i^2, \left(\frac{\sum Y_k}{N}\right)^2] \approx Cov[Y_i^2,Y_jY_k] \nonumber \\
  &Cov[-2Y_i\left(\frac{\sum Y_k}{N}\right), Y_j^2] \approx -2Cov[Y_i^2,Y_jY_k] \nonumber \\
  &Cov[-2Y_i\left(\frac{\sum Y_k}{N}\right),-2Y_j\left(\frac{\sum Y_k}{N}\right)] 
  \approx 4Cov[Y_iY_j,Y_kY_l] \nonumber \\
  &Cov[-2Y_i\left(\frac{\sum Y_k}{N}\right), \left(\frac{\sum Y_k}{N}\right)] 
  \approx -2Cov[Y_iY_j,Y_kY_l] \nonumber \\
  &Cov[\left(\frac{\sum Y_k}{N}\right)^2, Y_j^2] 
  \approx Cov[Y_j^2,Y_iY_k] \nonumber \\
  &Cov[\left(\frac{\sum Y_k}{N}\right)^2,-2Y_j\left(\frac{\sum Y_k}{N}\right)] 
  \approx -2Cov[Y_iY_j,Y_kY_l] \nonumber \\
  &Cov[\left(\frac{\sum Y_k}{N}\right)^2,\left(\frac{\sum Y_k}{N}\right)^2] 
  \approx Cov[Y_iY_j,Y_kY_l] \nonumber
\end{align*}

Combining all these gives 
\begin{equation}
  Cov[(Y_i - \frac{\sum Y_k}{N})^2, (Y_j - \frac{\sum Y_k}{N})^2] = 
  E[Y_i^2Y_j^2] - E[Y_i^2]^2 - 2E[Y_i^2Y_jY_k] + 2E[Y_i^2]E[Y_jY_k] +
  E[Y_iY_jY_kY_l] - E[Y_iY_j]^2.
\end{equation}

What we need to do next is derive expressions for each part of this covariance.